![UNIR](https://www.unir.net/wp-content/uploads/2014/10/logo.png)
### Máster en Inteligencia Artificial. 
**Asignatura:** _Aprendizaje Automático._

**Equipo 14:** Sergio Merino,  Laia García, Luisa Sánchez y Miguel Á. de Frutos

**Fecha:** _17 Enero 2020_

---

# ACTIVIDAD 2: Árboles y Random-Forest para regresión.

**Objetivo:** Aplicar al conjunto de datos proporcionado los métodos o algoritmos Árboles y Random Forest para predecir (regresión) el número de ventas de bicicletas por hora..
___

## 1. Análisis descriptivo de los datos.

In [1]:
#Importamos los paquetes principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Abrimos el dataset
rent_bike = pd.read_csv("./datos/hour.csv") 
#Exploramos los datos
print("Tamaño tabla:",rent_bike.shape)
rent_bike.head()   

Tamaño tabla: (17379, 17)


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [3]:
#Analizo tipo número de las variables
rent_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
instant       17379 non-null int64
dteday        17379 non-null object
season        17379 non-null int64
yr            17379 non-null int64
mnth          17379 non-null int64
hr            17379 non-null int64
holiday       17379 non-null int64
weekday       17379 non-null int64
workingday    17379 non-null int64
weathersit    17379 non-null int64
temp          17379 non-null float64
atemp         17379 non-null float64
hum           17379 non-null float64
windspeed     17379 non-null float64
casual        17379 non-null int64
registered    17379 non-null int64
cnt           17379 non-null int64
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


## 2. Tratamiento de datos: Missing y Redundancia.

- ***Missing:*** Analizamos si faltan datos o hay navores no numéricos.

In [4]:
#Buscamos NaN o Null para Pandas es lo mismo.
print(rent_bike.isnull().any())

instant       False
dteday        False
season        False
yr            False
mnth          False
hr            False
holiday       False
weekday       False
workingday    False
weathersit    False
temp          False
atemp         False
hum           False
windspeed     False
casual        False
registered    False
cnt           False
dtype: bool


- **Redundancias:** Elimino columnas no útiles o problemáticas. Ver razonamiento en el informe.

In [5]:
#Elimino variables que no deseo
rent_bike= rent_bike.drop(['casual' , 'registered','atemp','instant','dteday', 'season', 'holiday'], axis=1)
#Compruebo mi tabla reducida
rent_bike.head()  

,yr,mnth,hr,weekday,workingday,weathersit,temp,hum,windspeed,cnt
0,0,1,0,6,0,1,0.24,0.81,0.0,16
1,0,1,1,6,0,1,0.22,0.80,0.0,40
2,0,1,2,6,0,1,0.22,0.80,0.0,32
3,0,1,3,6,0,1,0.24,0.75,0.0,13
4,0,1,4,6,0,1,0.24,0.75,0.0,1


## 3. Tratamiento de variables categóricas.

In [6]:
#One-hot encoding para tratar variables categóricas.

## 4. Determinar el conjunto de modelización y el de validación.

In [7]:
from sklearn.model_selection import train_test_split 

datos_x = rent_bike.iloc[:,0:9]  #Variables entrada-features
datos_y = rent_bike.iloc[:,9]    #Variable respuesta                 

#Divido los datos en 80% entrenamiento y 20% test.
x_train, x_test, y_train, y_test = train_test_split(datos_x , datos_y, test_size=0.20, random_state=50) 


## 5. Identificar qué métricas de evaluación se utilizarán.

Vamos a usar 2 métricas para evaluar ambos algoritmos:

- **R^2 Score**: También llamada **"coeficinete de determinación"** o **"bondad del ajuste"**. Mide cómo de bien la variable respuesta explica las variables predictoras. 

    Lo mediremos como: **r2_score**(y_test, y_predicha)

        
- **MSE**: **"Error Cuadrático Medio"** recoge el promedio de los errores, es decir, la diferencia entre los valores predichos y por el modelo y los valores reales. 

    Lo mediremos como: **sqrt**(**mean_squared_error**(y_test, y_predicha))

In [27]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

## 6. Implementación de ambos algoritmos.

In [28]:
# Árbol de Decisión
from sklearn.tree import DecisionTreeRegressor

dec_Tree = DecisionTreeRegressor(random_state = 0)
dec_Tree.fit(x_train, y_train)
y_predicha = dec_Tree.predict(x_test)

rmse = sqrt(mean_squared_error(y_test, y_predicha))
r = r2_score(y_test, y_predicha)

print("RMSE:",rmse)
print("R^2 score:",r)

RMSE: 61.406827594811624
R^2 score: 0.8858427475869602


In [29]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor

rand_forest = RandomForestRegressor(random_state=0, n_estimators=300)
rand_forest.fit(x_train,y_train);
y_predicha = rand_forest.predict(x_test)

rmse = sqrt(mean_squared_error(y_test, y_predicha))
r = r2_score(y_test, y_predicha)

print("RMSE:",rmse)
print("R^2 score:",r)

RMSE: 45.776675413364764
R^2 score: 0.9365606563202391


## 7. Comparar los resultados obtenidos.

## 8. Otras consideraciones.

## 9. Repetir el ejecicio emplando un algoritmo de clasificación

In [10]:
# Preparar los datos
from sklearn.model_selection import train_test_split

clf_rent_bike = rent_bike.copy()

clf_data_y = []
for cnt in clf_rent_bike['cnt']:
    clf_data_y.append(1 if (cnt <= 20) else 2)

clf_data_x = clf_rent_bike.drop('cnt', axis=1)

#Divido los datos en 80% entrenamiento y 20% test.
clf_x_train, clf_x_test, clf_y_train, clf_y_test = train_test_split(clf_data_x , clf_data_y, test_size=0.20, random_state=50) 

print(f'Training data size: {len(clf_x_train)}')
print(f'Test data size: {len(clf_x_test)}')


Training data size: 13903
Test data size: 3476


In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

classification_tree = DecisionTreeClassifier(max_depth=2)
classification_tree.fit(clf_x_train, clf_y_train)

# Hacer la predicción
clf_prediction = classification_tree.predict(clf_x_test)

# Crear matriz de confusión
conf_matrix = confusion_matrix(clf_y_test, clf_prediction, labels=[1, 2])
accuracy = (conf_matrix[0][0] + conf_matrix[1][1]) / (conf_matrix[0][0] + conf_matrix[0][1] + conf_matrix[1][0] + conf_matrix[1][1])
print(f'Accuracy: {accuracy}')
print(f'Confusion matrix:\n     | <=20| >20|\n <=20|  {conf_matrix[0][0]}| {conf_matrix[1][0]}|\n  >20|   {conf_matrix[0][1]}|{conf_matrix[1][1]}|')



Accuracy: 0.9188722669735327
Confusion matrix:
     | <=20| >20|
 <=20|  504| 184|
  >20|   98|2690|
